In [1]:
# Run this first cell in Colab
!pip install -q chromadb sentence-transformers PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.5 MB/s et

In [2]:
import os
import math
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import PyPDF2
import numpy as np

# detect GPU if available (Colab sometimes provides GPU)
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [3]:



from google.colab import files
uploaded = files.upload()        # a file picker will open
pdf_local_path = list(uploaded.keys())[0]
print("Uploaded:", pdf_local_path)


Saving merge files.pdf to merge files.pdf
Uploaded: merge files.pdf


In [4]:
def extract_text_by_page(pdf_path):
    reader = PyPDF2.PdfReader(pdf_path)
    pages_text = []
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text is None:
            text = ""   # some pages may extract empty if complex layout
        pages_text.append((i + 1, text))  # store (page_number, text)
    return pages_text

pages = extract_text_by_page(pdf_local_path)
print(f"Extracted text from {len(pages)} pages.")
# optional: show first 500 chars of page 1
if pages:
    print("Page 1 sample:\n", pages[0][1][:500])


Extracted text from 124 pages.
Page 1 sample:
 Page 1 of 
117  
 
THE AGRICULTURAL PRODUCE (GRADING AND MARKING) 
ACT, 1937 
CONTENTS 
1. Short title and extent 
2. Explanations 
3. Prescription of grade designations 
4. Penalty for unauthorized or incorrect marking with grade designation mark 
5. Penalty for counterfeiting grade designation mark 
6. Extension of application of Act 
THE SCHEDULE 



In [5]:
def chunk_text(text, chunk_size=1000, overlap=200):
    """
    Simple character-based sliding window chunker.
    chunk_size: approx characters per chunk (tune as needed)
    overlap: overlapping characters between successive chunks
    """
    if not text:
        return []
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + chunk_size, text_len)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Build lists of chunks, ids and metadata
all_chunks = []
all_ids = []
all_metadatas = []
doc_id = os.path.basename(pdf_local_path)

for page_no, page_text in pages:
    page_chunks = chunk_text(page_text, chunk_size=1000, overlap=200)
    for i, c in enumerate(page_chunks):
        chunk_id = f"{doc_id}_p{page_no}_c{i}"
        all_chunks.append(c)
        all_ids.append(chunk_id)
        all_metadatas.append({"source_doc": doc_id, "page": page_no, "chunk_index": i})

print("Total chunks created:", len(all_chunks))


Total chunks created: 429


In [6]:
# Load the exact HF model you requested:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

print("Loading model:", MODEL_NAME)
model = SentenceTransformer(MODEL_NAME, device=device)

# Batch embed (safer for memory)
def embed_texts(model, texts, batch_size=64):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        emb = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        # ensure lists of floats for Chroma
        embeddings.extend([e.tolist() for e in emb])
    return embeddings

embeddings = embed_texts(model, all_chunks, batch_size=64)
print("Computed embeddings for all chunks. Example vector length:", len(embeddings[0]) if embeddings else 0)


Loading model: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computed embeddings for all chunks. Example vector length: 384


In [7]:
# Where to persist the DB (change to a Drive path if you mounted Drive)
PERSIST_DIR = "./chroma_db"

client = chromadb.PersistentClient(path=PERSIST_DIR)

COLLECTION_NAME = "pdf_documents"

# create_collection will error if name exists; use get_collection if it exists already
try:
    collection = client.create_collection(name=COLLECTION_NAME)
    print("Created new collection:", COLLECTION_NAME)
except Exception as e:
    print("Collection likely exists, getting it.")
    collection = client.get_collection(name=COLLECTION_NAME)

# Add data with precomputed embeddings
collection.add(
    ids=all_ids,
    documents=all_chunks,
    metadatas=all_metadatas,
    embeddings=embeddings
)

# persist to disk so it survives runtime restarts (as long as the folder is persisted or downloaded)
# client.persist() # persist is not needed with PersistentClient
print("Data added and persisted to:", PERSIST_DIR)

Created new collection: pdf_documents
Data added and persisted to: ./chroma_db


In [8]:
def semantic_search(query, n_results=3):
    # embed query using same model
    q_emb = model.encode([query], convert_to_numpy=True).tolist()
    # Use collection.query — this uses an embedding vector directly
    results = collection.query(
        query_embeddings=q_emb,
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

q = "Explain the main conclusion"   # change to any question relevant to your PDF
res = semantic_search(q, n_results=5)

# Pretty print the top results
for i in range(len(res["ids"][0])):
    print("---- RESULT", i+1, "----")
    print("ID:", res["ids"][0][i])
    print("Page:", res["metadatas"][0][i].get("page"))
    print("Distance:", res["distances"][0][i])
    print("Text snippet:\n", res["documents"][0][i][:500])
    print()

---- RESULT 1 ----
ID: merge files.pdf_p48_c0
Page: 48
Distance: 1.435021996498108
Text snippet:
 Page 3 of 
117  T
H
E
 
S
E
C
O
N
D
 
S
C
H
E
D
U
L
E
.
[
O
m
i
t
t
e
d
.
]

---- RESULT 2 ----
ID: merge files.pdf_p61_c0
Page: 61
Distance: 1.4590868949890137
Text snippet:
 Page 6 of 
117  
8. Order granting loan conclusive on certain points. A written order under the hand of an 
officer empowered to make loans under this Act granting a loan to, or with the consent of, a person 
mentioned therein, for the purpose of carrying out a work described therein, for the benefit of land 
specified therein, shall, for the purposes of this Act, be conclusive evidence- 
(a) that the work described is an improvement within the meaning of this Act; 
(b) that the person mentioned

---- RESULT 3 ----
ID: merge files.pdf_p22_c0
Page: 22
Distance: 1.4696296453475952
Text snippet:
 Page 11 of 
117  
l[the Court, if the amount of the creditor’s claim is disputed, shall examine both the plaintiff 
and the d

In [9]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.0 MB/s eta 0:00:00


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Pick a small but usable model (can change to a bigger one if GPU allows)
GEN_MODEL = "HuggingFaceH4/zephyr-7b-alpha"   # ~2.7B parameters

tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
generator_model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL,
    device_map="auto",     # uses GPU if available
    torch_dtype="auto"
)

# Create a text-generation pipeline
generator = pipeline(
    "text-generation",
    model=generator_model,
    tokenizer=tokenizer,
    max_new_tokens=300
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
